In [5]:
import xarray as xr
import os, glob

In [2]:
def preprocess(ds):
    ds = ds.expand_dims(west_east=1, south_north=1)
    ds = ds.assign_coords(west_east=ds.west_east, south_north=ds.south_north)
    return ds

In [17]:
client.close()
cluster.close()

In [3]:
import dask.distributed as dd
cluster = dd.LocalCluster(n_workers=24, dashboard_address=':22722')
client = dd.Client(cluster)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:22722/status,
Dashboard: http://127.0.0.1:22722/status,Workers: 24
Total threads: 96,Total memory: 187.55 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:41351,Workers: 24
Dashboard: http://127.0.0.1:22722/status,Total threads: 96
Started: Just now,Total memory: 187.55 GiB
Comm: tcp://127.0.0.1:45629,Total threads: 4
Dashboard: http://127.0.0.1:42477/status,Memory: 7.81 GiB
Nanny: tcp://127.0.0.1:33413,


In [7]:
root_dir = '/media/harish/SSD_4TB/EU_SCORES_project'
run = 'New_runs' #sys.argv[1]
case = 'Ireland_coast' #sys.argv[2]
level = 10 #int(sys.argv[3])
run_dir=f'{root_dir}/WRFV4.4/EU_SCORES/{run}/{case}/Postprocessed/variablewise_files'
target_dir=f'{run_dir}/weibull_{level}'
xr.open_dataset(f'{run_dir}/weibull_{level}.nc')

<xarray.Dataset>
Dimensions:         (hour: 24, month: 12, year: 31, parameter: 2,
                     south_north: 127, west_east: 127)
Coordinates:
  * hour            (hour) int64 0 1 2 3 4 5 6 7 8 ... 16 17 18 19 20 21 22 23
  * month           (month) int64 1 2 3 4 5 6 7 8 9 10 11 12
  * year            (year) int64 1990 1991 1992 1993 ... 2017 2018 2019 2020
  * parameter       (parameter) object 'shape' 'scale'
  * south_north     (south_north) int64 0 1 2 3 4 5 ... 121 122 123 124 125 126
  * west_east       (west_east) int64 0 1 2 3 4 5 6 ... 121 122 123 124 125 126
    XLONG           (south_north, west_east) float32 ...
    XLAT            (south_north, west_east) float32 ...
Data variables:
    hourly_values   (west_east, south_north, parameter, hour) float64 ...
    monthly_values  (west_east, south_north, parameter, month) float64 ...
    yearly_values   (west_east, south_north, parameter, year) float64 ...
    overall_values  (west_east, south_north, parameter) float64 ...

In [7]:
files = [f'{target_dir}/{i}.nc' for i in range(4)]
ds = xr.open_mfdataset(files,combine='nested', parallel=True, concat_dim='south_north')
ds

<xarray.Dataset>
Dimensions:         (hour: 24, month: 12, year: 31, parameter: 2,
                     south_north: 4, west_east: 63)
Coordinates:
  * hour            (hour) int64 0 1 2 3 4 5 6 7 8 ... 16 17 18 19 20 21 22 23
  * month           (month) int64 1 2 3 4 5 6 7 8 9 10 11 12
  * year            (year) int64 1990 1991 1992 1993 ... 2017 2018 2019 2020
  * parameter       (parameter) object 'shape' 'scale'
  * south_north     (south_north) int64 0 1 2 3
  * west_east       (west_east) int64 0 1 2 3 4 5 6 7 ... 56 57 58 59 60 61 62
Data variables:
    hourly_values   (west_east, south_north, parameter, hour) float64 dask.array<chunksize=(63, 1, 2, 24), meta=np.ndarray>
    monthly_values  (west_east, south_north, parameter, month) float64 dask.array<chunksize=(63, 1, 2, 12), meta=np.ndarray>
    yearly_values   (west_east, south_north, parameter, year) float64 dask.array<chunksize=(63, 1, 2, 31), meta=np.ndarray>
    overall_values  (west_east, south_north, parameter) float64 dask.array<chunksize=(63, 1, 2), meta=np.ndarray>

In [9]:
run_dir=f'{root_dir}/WRFV4.4/EU_SCORES/{run}/{case}/Postprocessed/variablewise_files'
XLAND = xr.open_dataset(f'{run_dir}/XLAND.nc')
XLAT = XLAND.XLAT
XLONG = XLAND.XLONG

In [20]:
ds = ds.assign_coords(XLAT=XLAT, XLONG=XLONG)
ds

<xarray.Dataset>
Dimensions:         (hour: 24, month: 12, year: 31, parameter: 2,
                     south_north: 63, west_east: 63)
Coordinates:
  * hour            (hour) int64 0 1 2 3 4 5 6 7 8 ... 16 17 18 19 20 21 22 23
  * month           (month) int64 1 2 3 4 5 6 7 8 9 10 11 12
  * year            (year) int64 1990 1991 1992 1993 ... 2017 2018 2019 2020
  * parameter       (parameter) object 'shape' 'scale'
  * south_north     (south_north) int64 0 1 2 3 4 5 6 7 ... 56 57 58 59 60 61 62
  * west_east       (west_east) int64 0 1 2 3 4 5 6 7 ... 56 57 58 59 60 61 62
    XLONG           (south_north, west_east) float32 7.657 7.665 ... 8.137 8.145
    XLAT            (south_north, west_east) float32 54.06 54.06 ... 54.34 54.34
Data variables:
    hourly_values   (west_east, south_north, parameter, hour) float64 dask.array<chunksize=(63, 63, 2, 24), meta=np.ndarray>
    monthly_values  (west_east, south_north, parameter, month) float64 dask.array<chunksize=(63, 63, 2, 12), meta=np.ndarray>
    yearly_values   (west_east, south_north, parameter, year) float64 dask.array<chunksize=(63, 63, 2, 31), meta=np.ndarray>
    overall_values  (west_east, south_north, parameter) float64 dask.array<chunksize=(63, 63, 2), meta=np.ndarray>